To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + support us if you can!
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

See on our [blog post](https://unsloth.ai/blog/gemma) on how we made **Gemma 7b 2.5x faster** and **Gemma 2x faster**!

In [1]:
# %%capture
import torch
import os

os.environ["WANDB_PROJECT"] = "gemma_2b_test"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
# major_version, minor_version = torch.cuda.get_device_capability()
# if major_version >= 8:
#     # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
#     !pip install "unsloth[colab-ampere] @ git+https://github.com/unslothai/unsloth.git"
# else:
#     # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
#     !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
# pass

* We support Llama, Mistral, CodeLlama, TinyLlama, Vicuna, Open Hermes etc
* And Yi, Qwen ([llamafied](https://huggingface.co/models?sort=trending&search=qwen+llama)), Deepseek, all Llama, Mistral derived archs.
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

/home/yashas/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.2.1+cu121)
    Python  3.10.13 (you have 3.10.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


==((====))==  Unsloth: Fast Gemma patching release 2024.3
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.647 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.22.post7. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


/home/yashas/miniconda3/envs/pytorch_test/lib/python3.10/site-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.3 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `ChatML` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# load json dataset at input.json
dataset = load_dataset('json', data_files='../combined_output.json', split='train')
dataset = dataset.map(formatting_prompts_func, batched = True,)

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 1,
        # max_steps = ,
        num_train_epochs = 10,
        learning_rate = 4e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to="wandb",
    ),
)

/home/yashas/miniconda3/envs/pytorch_test/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/yashas/miniconda3/envs/pytorch_test/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [6]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.647 GB.
2.219 GB of memory reserved.


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,173 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 2,930
 "-____-"     Number of trainable parameters = 19,611,648
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashas-salankimatt (yashas-personal). Use `wandb login --relogin` to force relogin


  0%|          | 1/2930 [00:02<1:42:01,  2.09s/it]

{'loss': 0.8959, 'grad_norm': 0.11932601034641266, 'learning_rate': 0.0004, 'epoch': 0.0}


  0%|          | 2/2930 [00:03<1:30:38,  1.86s/it]

{'loss': 0.9509, 'grad_norm': 0.12995079159736633, 'learning_rate': 0.000399863434619324, 'epoch': 0.01}


...

100%|█████████▉| 2929/2930 [1:17:14<00:01,  1.67s/it]

{'loss': 0.01, 'grad_norm': 0.14900048077106476, 'learning_rate': 1.3656538067599863e-07, 'epoch': 9.99}


100%|██████████| 2930/2930 [1:17:16<00:00,  1.67s/it]

{'loss': 0.0086, 'grad_norm': 0.13757388293743134, 'learning_rate': 0.0, 'epoch': 9.99}
{'train_runtime': 4642.9428, 'train_samples_per_second': 2.526, 'train_steps_per_second': 0.631, 'train_loss': 0.14177345379188963, 'epoch': 9.99}


100%|██████████| 2930/2930 [1:17:17<00:00,  1.58s/it]


In [8]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

4642.9428 seconds used for training.
77.38 minutes used for training.
Peak reserved memory = 19.908 GB.
Peak reserved memory for training = 17.689 GB.
Peak reserved memory % of max memory = 84.188 %.
Peak reserved memory for training % of max memory = 74.804 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [55]:
# import JSON file for evals
import json
with open('../eval_combined_output.json') as f:
    data = json.load(f)

print("Loaded", len(data), "examples.")


Loaded 186 examples.


In [56]:

# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

for i in range(len(data)):
    print("Parsing example", i+1, "out of", len(data))
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            data[i]['instruction'], # instruction
            data[i]['input'], # input
            ""
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
    model_output = tokenizer.batch_decode(outputs)
    response = model_output[0].replace(EOS_TOKEN, "")
    # print(response)
    response = response.split("### Response:")[1].strip()
    data[i]['predic'] = response
    # print(data[i]['predic'])
    # print(response)
    # print("=====================================================")

Parsing example 1 out of 186
Parsing example 2 out of 186
Parsing example 3 out of 186
Parsing example 4 out of 186
Parsing example 5 out of 186
Parsing example 6 out of 186
Parsing example 7 out of 186
Parsing example 8 out of 186
Parsing example 9 out of 186
Parsing example 10 out of 186
Parsing example 11 out of 186
Parsing example 12 out of 186
Parsing example 13 out of 186
Parsing example 14 out of 186
Parsing example 15 out of 186
Parsing example 16 out of 186
Parsing example 17 out of 186
Parsing example 18 out of 186
Parsing example 19 out of 186
Parsing example 20 out of 186
Parsing example 21 out of 186
Parsing example 22 out of 186
Parsing example 23 out of 186
Parsing example 24 out of 186
Parsing example 25 out of 186
Parsing example 26 out of 186
Parsing example 27 out of 186
Parsing example 28 out of 186
Parsing example 29 out of 186
Parsing example 30 out of 186
Parsing example 31 out of 186
Parsing example 32 out of 186
Parsing example 33 out of 186
Parsing example 34 

In [57]:
# output data to JSON file
with open('../eval_tokens_over_time.json', 'w') as f:
    json.dump(data, f, indent=2)

wandb: Network error (ConnectTimeout), entering retry loop.


In [41]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         "Continue the fibonnaci sequence.", # instruction
#         "1, 1, 2, 3, 5, 8", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Given  a language description of the task in the frame sequence and the following few token representations of a visual frame in the following format 'frame t-0: [{{object_id, {{centroid_x, centroid_y}},radial_points}}]', predict the next two frames in the following format 'frame t+0: [{{object_id, {{centroid_x, centroid_y}},radial_points}}]'",
        "put the red block on the yellow block\nframe t-2: [{gripper,{204,38},68,71,76,73,0,0,0,45,55,52,87,70,62,61,65,67,51,44,40,40,42,47,57,76,77}, {table,{309,308},331,342,353,249,202,179,171,174,188,221,290,334,313,313,334,296,274,250,212,193,190,198,223,274,342}, {yellow block,{265,265},34,29,27,27,28,33,41,44,44,45,45,40,37,30,26,24,24,26,30,39,43,43,47,44,40}, {green block,{165,217},43,39,32,28,27,28,31,35,42,41,41,40,41,42,32,26,24,24,24,27,32,40,42,44,43}, {blue block,{387,328},45,40,34,31,31,33,38,48,50,49,50,48,45,42,35,30,30,29,32,38,48,49,50,53,49}, {red block,{316,195},37,39,42,40,40,37,29,26,24,24,26,31,34,36,38,38,37,36,30,27,24,23,25,27,34}]\nframe t-1: [{gripper,{205,38},67,71,76,74,0,0,0,45,56,54,87,70,62,61,66,67,51,44,40,40,42,47,57,75,77}, {table,{309,308},331,342,353,249,202,180,171,174,188,221,290,334,313,313,334,295,273,249,211,192,189,198,223,274,342}, {yellow block,{266,265},33,29,27,27,28,32,41,44,45,46,45,40,37,30,25,24,24,26,30,39,43,43,47,44,40}, {green block,{166,218},42,38,30,28,27,28,31,35,42,41,41,40,41,44,33,26,24,24,25,27,32,41,42,44,43}, {blue block,{387,328},46,40,35,32,31,34,39,48,50,49,50,48,45,42,33,30,29,29,31,38,47,49,50,53,49}, {red block,{317,196},36,39,40,40,38,37,29,26,24,24,26,30,34,36,39,40,38,36,31,26,23,23,24,28,35}]\nframe t-0: [{gripper,{207,42},72,69,75,77,4,4,4,52,62,58,89,74,65,62,66,72,56,48,44,44,46,51,63,74,73}, {table,{309,308},331,342,353,250,202,180,172,175,189,221,290,334,313,313,334,295,274,247,212,193,189,198,223,274,342}, {yellow block,{266,266},33,28,27,27,28,31,39,43,44,45,45,40,37,31,26,24,26,27,31,41,43,44,49,44,40}, {green block,{166,218},43,38,30,28,27,28,31,35,42,41,41,40,41,44,32,25,24,24,25,27,31,41,41,44,43}, {blue block,{388,328},45,40,34,31,31,33,38,48,51,50,50,49,46,43,34,30,29,29,31,38,48,49,50,53,48}, {red block,{317,196},36,38,40,40,38,37,30,26,25,25,26,30,34,36,38,40,38,37,31,27,23,23,24,28,35}]",
        ""
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
tokenizer.batch_decode(outputs)

["<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGiven  a language description of the task in the frame sequence and the following few token representations of a visual frame in the following format 'frame t-0: [{{object_id, {{centroid_x, centroid_y}},radial_points}}]', predict the next two frames in the following format 'frame t+0: [{{object_id, {{centroid_x, centroid_y}},radial_points}}]'\n\n### Input:\nput the red block on the yellow block\nframe t-2: [{gripper,{204,38},68,71,76,73,0,0,0,45,55,52,87,70,62,61,65,67,51,44,40,40,42,47,57,76,77}, {table,{309,308},331,342,353,249,202,179,171,174,188,221,290,334,313,313,334,296,274,250,212,193,190,198,223,274,342}, {yellow block,{265,265},34,29,27,27,28,33,41,44,44,45,45,40,37,30,26,24,24,26,30,39,43,43,47,44,40}, {green block,{165,217},43,39,32,28,27,28,31,35,42,41,41,40,41,42,32,26,24,24,24,27,32,40,42

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [28]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         "Continue the fibonnaci sequence for the next 6 numbers.", # instruction
#         "1, 1, 2, 3, 5, 8, 13, 21", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

instruction = "a language description of the task in the frame sequence and the following few token representations of a visual frame in the following format 'frame t-0: [{{object_id, {{centroid_x, centroid_y}},radial_points}}]', predict the next two frames in the following format 'frame t+0: [{{object_id, {{centroid_x, centroid_y}},radial_points}}]'"

inputs = tokenizer(
[
    alpaca_prompt.format(
		instruction,
		"put the yellow object on top of the green object\nframe t-2: [{gripper,{224,121},68,70,75,91,167,188,116,145,90,18,13,11,11,11,230,116,159,135,125,123,129,109,86,80,79}, {table,{315,317},325,336,337,237,191,171,163,165,180,211,276,340,319,319,340,308,286,98,134,202,197,207,233,287,336}, {yellow block,{210,314},43,36,32,30,30,33,38,50,50,49,50,48,46,37,30,28,27,28,30,38,48,49,52,53,49}, {green block,{177,210},6,6,6,9,19,29,32,37,48,54,55,54,55,40,30,25,23,21,22,26,20,37,43,150,139}, {blue block,{387,328},46,40,34,31,31,34,39,48,50,49,50,49,46,41,33,30,28,28,30,36,47,49,50,54,49}, {red block,{345,197},40,42,43,41,40,32,28,25,24,25,29,36,39,39,41,39,38,34,28,25,24,24,27,31,41}]\nframe t-1: [{gripper,{218,127},82,75,79,94,170,192,125,151,92,11,9,9,9,9,236,106,167,142,131,129,124,111,90,79,82}, {table,{316,317},324,335,337,237,192,171,163,165,180,211,276,341,320,320,341,308,288,39,155,201,197,206,231,286,335}, {yellow block,{210,314},42,36,32,30,30,33,38,50,50,49,50,48,47,38,30,28,26,28,31,37,47,48,52,53,50}, {green block,{178,213},36,2,2,2,2,4,4,33,44,52,53,53,55,47,33,28,25,24,24,26,26,27,30,137,140}, {blue block,{387,328},46,40,34,32,31,34,39,47,50,49,50,49,46,41,33,30,29,28,30,36,47,49,50,54,49}, {red block,{346,197},39,41,42,40,40,32,28,26,25,26,29,36,41,38,36,39,38,34,28,24,23,23,25,30,40}]\nframe t-0: [{gripper,{210,132},83,75,83,95,172,196,129,155,92,10,8,8,8,8,227,106,173,147,136,132,120,109,94,80,81}, {table,{317,318},323,334,335,236,191,170,162,164,178,209,274,343,321,321,343,310,289,38,175,202,198,207,233,287,334}, {yellow block,{209,315},40,36,32,30,30,33,38,49,48,48,49,47,46,38,30,28,27,27,30,37,50,50,54,47,43}, {green block,{177,213},28,30,0,0,0,0,0,35,46,51,52,52,53,45,33,28,25,24,25,29,31,30,31,124,134}, {blue block,{387,328},45,40,34,31,31,33,39,47,50,49,50,48,46,41,33,29,29,28,30,36,47,49,50,54,49}, {red block,{345,197},40,41,43,40,40,32,28,25,24,26,29,35,38,38,40,40,38,34,28,25,23,24,27,31,41}]",
		""
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
a language description of the task in the frame sequence and the following few token representations of a visual frame in the following format 'frame t-0: [{{object_id, {{centroid_x, centroid_y}},radial_points}}]', predict the next two frames in the following format 'frame t+0: [{{object_id, {{centroid_x, centroid_y}},radial_points}}]'

### Input:
put the yellow object on top of the green object
frame t-2: [{gripper,{224,121},68,70,75,91,167,188,116,145,90,18,13,11,11,11,230,116,159,135,125,123,129,109,86,80,79}, {table,{315,317},325,336,337,237,191,171,163,165,180,211,276,340,319,319,340,308,286,98,134,202,197,207,233,287,336}, {yellow block,{210,314},43,36,32,30,30,33,38,50,50,49,50,48,46,37,30,28,27,28,30,38,48,49,52,53,49}, {green block,{177,210},6,6,6,9,19,29,32,37,48,54,55,54,55,40,30,25,23,21,22,26,20

frame t+1: [{gripper,{209,136},88,87,81,102,182,203,145,153,102,8,7,7,8,8,234,108,180,153,141,139,12,111,98,86,84}, {table,{318,319},322,333,333,234,190,169,161,163,177,208,273,344,322,322,344,312,291,0,0,203,198,208,233,287,333}, {yellow block,{210,316},39,35,31,29,29,31,37,48,47,47,48,47,47,39,30,28,26,27,29,37,47,47,50,46,44}, {green block,{173,219},24,28,32,32,32,35,33,29,26,26,25,24,25,27,30,28,28,29,33,35,30,25,24,23,22}, {blue block,{387,328},45,40,34,31,31,33,39,47,50,49,50,49,46,42,33,29,29,28,30,36,47,49,50,54,48}, {red block,{343,198},40,42,40,36,34,33,30,27,26,27,29,36,36,35,36,38,35,33,30,28,27,27,31,34,40}]<eos>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [11]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [12]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\nput the green block on top of the yellow block\n\n\n### Input:\nput the yellow block on top of the green block\n\n\n### Response:\nput the yellow block on top of the green block\n\n\n### Input:\nput the green block on top of the blue block\n\n\n### Response:\nput the yellow block']

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [13]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [14]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [15]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>